In [ ]:
!pip install accelerate

In [ ]:
import PIL
from PIL import Image
import numpy as np
import torch
from torch import autocast
from slugify import slugify
from time import time

from diffusers import DDIMScheduler, StableDiffusionImg2ImgPipeline
from diffusion_utils import stable_diffusion_preprocess_image

In [ ]:
device     = 'cuda'
# model_path = 'models/stable-diffusion-v1.4'
model_path = 'models/stable-diffusion-v1.5'

scheduler = DDIMScheduler(beta_start=0.00085, beta_end=0.012, beta_schedule="scaled_linear", clip_sample=False, set_alpha_to_one=False)
model = StableDiffusionImg2ImgPipeline.from_pretrained(
    model_path,
    scheduler=scheduler,
    revision='fp16',
    torch_dtype=torch.float16,
).to(device)

In [ ]:
model.safety_checker = (lambda images, clip_input: (images, False))

In [ ]:
def make_image_for_path(path, img_show):

    # prompt = 'a chicken explaining to chicks, equations on the blackboard, elegant, intricate, digital painting, artstation, concept art, smooth, sharp focus, illustration, art by artgerm and greg rutkowski and alphonse mucha'
    generator = torch.Generator(device=device).manual_seed(42)
    # prompt = "digital painting of Les Filles de Thespius by Gustave Moreau, very detailed faces"
    prompt = "masterwork digital painting of shrek in the style of John Everett Millais, trending on artstation"
    start_image_preprocessed = stable_diffusion_preprocess_image(path)
    repetitions = 1

    with autocast('cuda'):
        for r in range(repetitions):
            image = model(
                prompt = prompt,
                init_image = start_image_preprocessed,
                num_inference_steps=100,
                strength = 0.3, # 0 = no noise, 1 = max noise
                guidance_scale = 7.5,# how much to rely on the textual prompt
                generator = generator,
            )
            image = image.images[0]
            if img_show:
                image.show()
            image.save(f'images/{time()}_{slugify(prompt[:100])}.png')

In [ ]:
from glob import glob
input_path = "images/shrek_resized"
filename_list = sorted(glob(input_path+"/*", recursive = True))
# filename_list

In [ ]:
from tqdm.notebook import trange, tqdm

# resume from image no 1839

for index,curr_path in tqdm(enumerate(filename_list)):
    if index % 3 == 0:
        print("image number : " +str(index))
        make_image_for_path(curr_path, False)

# Convert movie extract to 512x512 fimages/shrek_resized/

In [ ]:
# from glob import glob
# input_path = "images/shrek_resized/"
# filename_list = sorted(glob(input_path+"/*", recursive = True))
# filename_list

In [ ]:
# from PIL import Image, ImageOps

# for curr_image_path in filename_list:
#     with Image.open(curr_image_path) as im:
#         im = ImageOps.pad(im, (512, 512), color='black')
#     im.save(curr_image_path)
#     # im.show()